In [25]:
import funzioni01 as f1
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.providers.ibmq.job import job_monitor
#from qiskit.providers.ibmq import least_busy
from copy import deepcopy
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
from qiskit.circuit import Parameter
import time
import qiskit.quantum_info as qi
from qiskit.utils import QuantumInstance
import qiskit.ignis.mitigation.measurement as mc
#from qiskit.result import marginal_counts
from sympy import *
from sympy.solvers import solve
from sympy import Symbol
#import openql as ql
from sympy.physics.quantum import TensorProduct as Tp
from sympy import sqrt, cos, sin, acos, asin, atan
import numpy as np
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute


In [26]:
e=Symbol("e", positive=True)

r_1 = Symbol("r1")#, positive = True)
phi_1 = Symbol("f1")#, positive = True)
a_1 = Symbol("a1")#, positive = True)
c_1 = Symbol("c1")#, positive = True)

r_2 = Symbol("r2")#, positive = True)
phi_2 = Symbol("f2")#, positive = True)
a_2 = Symbol("a2")#, positive = True)
c_2 = Symbol("c2")#, positive = True)

r_3 = Symbol("r3")#, positive = True)
phi_3 = Symbol("f3")#, positive = True)
a_3 = Symbol("a3")#, positive = True)
c_3 = Symbol("c3")#, positive = True)

r_4 = Symbol("r4")#, positive = True)
phi_4 = Symbol("f4")#, positive = True)
c_4 = Symbol("c5")#, positive = True)
a_4 = Symbol("c6")#, positive = True)

Id=eye(2)

cx_01= Matrix([
[1,0,0,0],
[0,0,0,1],
[0,0,1,0],
[0,1,0,0]
])
cx_10= Matrix([
[1,0,0,0],
[0,1,0,0],
[0,0,0,1],
[0,0,1,0]
])
m_1 = Matrix([
#[e**(1j*c_1),0,0,0],
#[0,e**(1j*c_1),0,0],
[1,0,0,0],
[0,1,0,0],
[0,0,e**(1j*a_1)*cos(r_1),e**(1j*phi_1)*sin(r_1)],
[0,0,-e**(-1j*phi_1)*sin(r_1),e**(-1j*a_1)*cos(r_1)]
])

m_2 = Matrix([
#[e**(1j*c_2),0,0,0],
#[0,e**(1j*c_2),0,0],
[1,0,0,0],
[0,1,0,0],
[0,0,e**(1j*a_2)*cos(r_2),e**(1j*phi_2)*sin(r_2)],
[0,0,-e**(-1j*phi_2)*sin(r_2),e**(-1j*a_2)*cos(r_2)]
])

m_3 = Matrix([
#[e**(1j*c_3),0,0,0],
#[0,e**(1j*c_3),0,0],
[1,0,0,0],
[0,1,0,0],
[0,0,e**(1j*a_3)*cos(r_3),e**(1j*phi_3)*sin(r_3)],
[0,0,-e**(-1j*phi_3)*sin(r_3),e**(-1j*a_3)*cos(r_3)]
])

m_4 = Matrix([
#[e**(1j*c_4),0,0,0],
#[0,e**(1j*c_4),0,0],
[1,0,0,0],
[0,1,0,0],
[0,0,e**(1j*a_4)*cos(r_4),e**(1j*phi_4)*sin(r_4)],
[0,0,-e**(-1j*phi_4)*sin(r_4),e**(-1j*a_4)*cos(r_4)]
])

y = Matrix([
[a_3+1j*c_1,a_2+1j*c_1,a_1-1j*2*c_1,0],
[a_1+1j*c_1,a_3-1j*2*c_1,a_2+1j*c_1,0],
[a_2-1j*2*c_1,a_1+1j*c_1,a_3+1j*c_1,0],
[0,0,0,1],
])

B = Matrix([
    [1,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
])
C= Matrix([
    [0,0,0,1,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [1,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
])
D= Matrix([
    [1,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
])

In [27]:
cx_01*m_4*cx_01

Matrix([
[1,                     0,                      0, 0],
[0, cos(r4)/e**(1.0*I*c6), -sin(r4)/e**(1.0*I*f4), 0],
[0, e**(1.0*I*f4)*sin(r4),  e**(1.0*I*c6)*cos(r4), 0],
[0,                     0,                      0, 1]])

In [28]:
y

Matrix([
[a3 + 1.0*I*c1, a2 + 1.0*I*c1, a1 - 2.0*I*c1, 0],
[a1 + 1.0*I*c1, a3 - 2.0*I*c1, a2 + 1.0*I*c1, 0],
[a2 - 2.0*I*c1, a1 + 1.0*I*c1, a3 + 1.0*I*c1, 0],
[            0,             0,             0, 1]])

In [29]:
M_1=Tp(Id,cx_01*m_1*cx_01)
M_2=Tp(cx_01*m_2*cx_01,Id)
M_3=Tp(Id,cx_01*m_3*cx_01)
M_4=Tp(cx_01*m_4*cx_01,Id)
#Swap=Tp(swap,Id)
#E_3=Swap*Tp(Id,cx_01*m_3*cx_01)*Swap
Y=Tp(Id,y)
D*C*Y*C*D

Matrix([
[1,             0,             0,             0,             0,             0,             0, 0],
[0, a3 - 2.0*I*c1, a1 + 1.0*I*c1, a2 + 1.0*I*c1,             0,             0,             0, 0],
[0, a2 + 1.0*I*c1, a3 + 1.0*I*c1, a1 - 2.0*I*c1,             0,             0,             0, 0],
[0, a1 + 1.0*I*c1, a2 - 2.0*I*c1, a3 + 1.0*I*c1,             0,             0,             0, 0],
[0,             0,             0,             0, a3 + 1.0*I*c1, a2 + 1.0*I*c1, a1 - 2.0*I*c1, 0],
[0,             0,             0,             0, a1 + 1.0*I*c1, a3 - 2.0*I*c1, a2 + 1.0*I*c1, 0],
[0,             0,             0,             0, a2 - 2.0*I*c1, a1 + 1.0*I*c1, a3 + 1.0*I*c1, 0],
[0,             0,             0,             0,             0,             0,             0, 1]])

In [30]:
rrrruuuben=M_2*M_1

In [31]:
precision=15
tempo=np.pi
steps=10

U=f1.Trotter_N_approx(steps=steps, tempo=tempo, precision=5)
U

Matrix([
[1.0 - 9.3738e-7*I,                     0,                      0,                      0,                      0,                      0,                     0,                 0],
[                0,  0.96673 + 0.088013*I, -0.079282 + 0.088013*I,                      0,    0.11255 - 0.17602*I,                      0,                     0,                 0],
[                0,  0.11255 + 0.088013*I,    0.96673 - 0.17602*I,                      0, -0.079282 + 0.088013*I,                      0,                     0,                 0],
[                0,                     0,                      0,   0.96673 + 0.088013*I,                      0,   0.11255 + 0.088013*I, -0.079282 - 0.17602*I,                 0],
[                0, -0.079282 - 0.17602*I,   0.11255 + 0.088013*I,                      0,   0.96673 + 0.088013*I,                      0,                     0,                 0],
[                0,                     0,                      0, -0.079282 + 0.

In [32]:
B=Matrix([
    [0,0,0,0,1,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [1,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
])

C=Matrix([
    [0,0,0,1,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [1,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
])

D=Matrix([
    [1,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
])

(B*C*D*U*D*C*B).evalf(3)


Matrix([
[  0.967 + 0.088*I,   0.113 + 0.088*I, -0.0793 - 0.176*I,               0,                 0,                 0,                 0,               0],
[-0.0793 + 0.088*I,   0.967 - 0.176*I,   0.113 + 0.088*I,               0,                 0,                 0,                 0,               0],
[  0.113 - 0.176*I, -0.0793 + 0.088*I,   0.967 + 0.088*I,               0,                 0,                 0,                 0,               0],
[                0,                 0,                 0, 1.0 - 9.37e-7*I,                 0,                 0,                 0,               0],
[                0,                 0,                 0,               0,   0.967 + 0.088*I,   0.113 + 0.088*I, -0.0793 - 0.176*I,               0],
[                0,                 0,                 0,               0, -0.0793 + 0.088*I,   0.967 - 0.176*I,   0.113 + 0.088*I,               0],
[                0,                 0,                 0,               0,   0.113 - 0.176*

In [33]:
B*C*D*rrrruuuben*D*C*B

Matrix([
[ e**(1.0*I*a2)*cos(r2), e**(1.0*I*a1)*e**(1.0*I*f2)*sin(r2)*cos(r1), e**(1.0*I*f1)*e**(1.0*I*f2)*sin(r1)*sin(r2), 0,                     0,                                              0,                                             0, 0],
[-sin(r2)/e**(1.0*I*f2), e**(1.0*I*a1)*cos(r1)*cos(r2)/e**(1.0*I*a2), e**(1.0*I*f1)*sin(r1)*cos(r2)/e**(1.0*I*a2), 0,                     0,                                              0,                                             0, 0],
[                     0,                      -sin(r1)/e**(1.0*I*f1),                       cos(r1)/e**(1.0*I*a1), 0,                     0,                                              0,                                             0, 0],
[                     0,                                           0,                                           0, 1,                     0,                                              0,                                             0, 0],
[                     0,       

In [20]:
M=B*C*D
M


Matrix([
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])

In [14]:
M**(-1)

Matrix([
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])

In [15]:
U2=Matrix([
    [U[3*8+3],U[3*8+5],U[3*8+6],0],
    [U[5*8+3],U[5*8+5],U[5*8+6],0],
    [U[6*8+3],U[6*8+5],U[6*8+6],0],
    [0,0,0,1]
])

M**(-1)*Tp(eye(2),U2)*M

Matrix([
[1,                     0,                      0,                      0,                      0,                      0,                     0, 0],
[0,  0.96673 + 0.088013*I, -0.079282 + 0.088013*I,                      0,    0.11255 - 0.17602*I,                      0,                     0, 0],
[0,  0.11255 + 0.088013*I,    0.96673 - 0.17602*I,                      0, -0.079282 + 0.088013*I,                      0,                     0, 0],
[0,                     0,                      0,   0.96673 + 0.088013*I,                      0,   0.11255 + 0.088013*I, -0.079282 - 0.17602*I, 0],
[0, -0.079282 - 0.17602*I,   0.11255 + 0.088013*I,                      0,   0.96673 + 0.088013*I,                      0,                     0, 0],
[0,                     0,                      0, -0.079282 + 0.088013*I,                      0,    0.96673 - 0.17602*I,  0.11255 + 0.088013*I, 0],
[0,                     0,                      0,    0.11255 - 0.17602*I,                 

In [16]:
eta_0=im(U[8*3+3])
alpha_1=re(U[8*5+3])
alpha_2=re(U[8*6+3])
alpha_2

0.11255

In [17]:
R2=acos(sqrt((1-alpha_1-alpha_2)**2+eta_0**2))
C1=atan(eta_0/(1-alpha_1-alpha_2))
A2=A1=0
R1=acos(sqrt(alpha_1**2+eta_0**2)/sin(R2))
F2=atan(eta_0/alpha_1)
F1=atan(-2*eta_0/alpha_2)-F2
sin(R1)*sin(R2)
sqrt(alpha_1**2+4*eta_0**2)
F1+F2
atan(-2*eta_0/alpha_1)


1.1476

In [18]:
M_1=M_1.subs(r_1,R1)
M_1=M_1.subs(a_1,A1)
M_1=M_1.subs(phi_1,F1)
M_1=M_1.subs(c_1,C1)
M_1=M_1.subs(e,np.e)

M_2=M_2.subs(r_2,R2)
M_2=M_2.subs(a_2,A2)
M_2=M_2.subs(phi_2,F2)
M_2=M_2.subs(c_2,C1)
M_2=M_2.subs(e,np.e)

M_1=M_1.evalf(precision)
M_2=M_2.evalf(precision)

In [ ]:
MM=(M_1*M_2).evalf(precision)
MM

Matrix([
[0.996229906141067 + 0.0867523723604232*I,                                        0,                                         0,                                         0,                                          0,                                          0,                                        0,                                        0],
[                                       0, 0.478073459497706 + 0.0207761674376247*I,  -0.859152524711035 - 0.156159449748112*I,                                         0,  0.0320636624358665 + 0.0863551114799732*I,                                          0,                                        0,                                        0],
[                                       0,  0.869922512207742 - 0.119368927886073*I,                         0.475884241171202,                                         0, -0.0256079966949834 - 0.0431775557399873*I,                                          0,                                        0,   

In [ ]:
r = Symbol("r")  # , positive = True)

MMM = Matrix([
    [0, r],
    [2*r, 0]
])

MMM = MMM.subs(r, -1-1j)
MMM.evalf(10)


Matrix([
[           0, -1.0 - 1.0*I],
[-2.0 - 2.0*I,            0]])

In [ ]:
x=atan(eta_0/(1-alpha_1-alpha_2))
y=atan(eta_0/alpha_2)
z=atan(-2*eta_0/alpha_1)

equazioni=[
    -a_1-a_3+c_3-x,
    -a_3+c_3+phi_1-y,
    c_1+c_3+phi_3-z,
    np.pi-a_3+c_2-phi_2-y,
    np.pi+a_3+c_4-phi_1-y,
    a_3+c_1+phi_2-y,
    -a_2-a_3+c_2-x,
    a_2+a_3+c_1-x,
    np.pi+c_2+c_4-phi_3-z,
    a_1+a_3+c_4-x,
    #c_1+c_3+c_5,
    #c_2+c_4+c_6
]
#a_1,a_2,a_3,phi_1,phi_2,phi_3

In [ ]:
solve(equazioni,[a_1,a_2,a_3,phi_1,phi_2,phi_3,c_1])

[]

In [ ]:
x=atan(-2*eta_0/alpha_1)
y=atan(eta_0/(1-alpha_1-alpha_2))
z=atan(-2*eta_0/alpha_2)
w=atan(eta_0/alpha_1)
k=atan(eta_0/alpha_2)


equazioni=[
    c_1+phi_2+phi_3-x,
    -a_2+c_2+c_6-y,
    c_6+phi_1+phi_2-z,
    np.pi+a_1+c_5-phi_2-w,
    np.pi-a_3+c_2-phi_2-k,
    a_3+c_1+phi_2-k,
    -a_1+c_6+phi_2-w,
    c_5-phi_1-phi_2-z,
    a_2+c_1+c_5-y,
    c_2-phi_2-phi_3-x,
    #c_1+c_3+c_5,
    #c_2+c_4+c_6
]

In [ ]:
solve(equazioni,[a_1,a_2,a_3])

[]

In [ ]:
from sympy import Matrix, solve_linear_system
Eqs_matrix=Matrix((
    #a_1,a_2,a_3,f_1,f_2,f_3
    (0,0,0,0,1,1,1,0,0,0,0,0,x),
    (0,-1,0,0,0,0,0,1,0,0,0,1,y),
    (0,0,0,1,1,0,0,0,0,0,0,1,z),
    (1,0,0,0,-1,0,0,0,0,0,1,0,w-np.pi),
    (0,0,-1,0,-1,0,0,1,0,0,0,0,k-np.pi),
    (0,0,1,0,1,0,1,0,0,0,0,0,k),
    (-1,0,0,0,1,0,0,0,0,0,0,1,w),
    #(0,0,0,-1,-1,0,0,0,0,0,1,0,z),
    #(0,1,0,0,0,0,1,0,0,0,1,0,y),
    #(0,0,0,0,-1,-1,0,1,0,0,0,0,x),
    #(0,0,0,0,0,0,1,0,1,0,1,0,0),
    #(0,0,0,0,0,0,0,1,0,1,0,1,0)
))

In [ ]:
sol=solve_linear_system(Eqs_matrix,a_1,a_2,a_3,phi_1,phi_2,phi_3,c_1,c_2,c_3,c_4,c_5,c_6)
print(sol)

None


In [ ]:
a,b,c,d=symbols("a b c d")
M=Matrix([
    [a,b],
    [c,d]
])

Tp(eye(2),M)

Matrix([
[a, b, 0, 0],
[c, d, 0, 0],
[0, 0, a, b],
[0, 0, c, d]])